In [1]:
%%capture
%pip install -e ../../..

In [2]:
import os
print(os.path.exists('path/to/your/lmexp/models/implementations/llama3.py'))

False


In [3]:
import os
print(os.getcwd())

/root/lmexp/lmexp/notebooks


In [4]:
import os
print(os.path.exists('../llama3.py'))

False


In [5]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

list_files('.')

./
    gpt2small.ipynb
    .ipynb_checkpoints/
        gpt2small-checkpoint.ipynb


In [6]:
import os
print("GPT2 file exists:", os.path.exists('./models/implementations/gpt2small.py'))
print("Llama3 file exists:", os.path.exists('./models/implementations/llama3.py'))

GPT2 file exists: False
Llama3 file exists: False


In [7]:
import lmexp.models.implementations.gpt2small as gpt2
print(gpt2.__file__)

ImportError: libcudnn.so.8: cannot open shared object file: No such file or directory

In [8]:
import sys
print(sys.path)

['/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages/accelerate-0.31.0-py3.10.egg', '/opt/conda/lib/python3.10/site-packages/ipykernel-6.29.4-py3.10.egg', '/opt/conda/lib/python3.10/site-packages/transformers-4.41.2-py3.10.egg', '/opt/conda/lib/python3.10/site-packages/torch-2.3.1-py3.10-linux-x86_64.egg', '/opt/conda/lib/python3.10/site-packages/requests-2.32.3-py3.10.egg', '/opt/conda/lib/python3.10/site-packages/python_dotenv-1.0.1-py3.10.egg', '/opt/conda/lib/python3.10/site-packages/numpy-2.0.0-py3.10-linux-x86_64.egg', '/opt/conda/lib/python3.10/site-packages/bitsandbytes-0.42.0-py3.10.egg', '/opt/conda/lib/python3.10/site-packages/anthropic-0.29.0-py3.10.egg', '/opt/conda/lib/python3.10/site-packages/safetensors-0.4.3-py3.10-linux-x86_64.egg', '/opt/conda/lib/python3.10/site-packages/huggingface_hub-0.23.4-py3.10.egg', '/opt/conda/lib/python3.

In [11]:
import importlib
import lmexp.models.implementations.gpt2small as gpt2
importlib.reload(gpt2)

<module 'lmexp.models.implementations.gpt2small' from '/opt/conda/lib/python3.10/site-packages/lmexp-0.1-py3.10.egg/lmexp/models/implementations/gpt2small.py'>

In [11]:
echo $LD_LIBRARY_PATH

SyntaxError: invalid syntax (2492339924.py, line 1)

In [10]:
import torch
print(torch.__version__)
print(torch.version.cuda)

ImportError: libcudnn.so.8: cannot open shared object file: No such file or directory

In [9]:
from lmexp.models.implementations.gpt2small import GPT2Tokenizer
print(GPT2Tokenizer.__module__)

ImportError: libcudnn.so.8: cannot open shared object file: No such file or directory

In [17]:
import lmexp
print(lmexp.__file__)

/opt/conda/lib/python3.10/site-packages/lmexp-0.1-py3.10.egg/lmexp/__init__.py


In [15]:
from lmexp.models.implementations.gpt2small import GPT2Tokenizer, ProbedGPT2
from lmexp.models.implementations.llama3 import Llama3Tokenizer, ProbedLlama3
from lmexp.generic.probing import train_probe
from lmexp.generic.caa import get_caa_vecs
from lmexp.generic.hooked_model import run_simple_steering
from datetime import datetime
import random

ModuleNotFoundError: No module named 'lmexp.models.implementations.llama3'

# Load GPT2 model and tokenizer

These classes have already implemented all the probing-related methods so we won't have to add more hooks + they are ready to use with our vector extraction and steering functions.

In [3]:
model = ProbedGPT2()
tokenizer = GPT2Tokenizer()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
model.get_n_layers()

12

# Load Llama 3 model and tokenizer


In [21]:
model = ProbedLlama3()

NameError: name 'ProbedLlama3' is not defined

In [ ]:
tokenizer = Llama3Tokenizer()

# Training a linear probe

## Generate some data

Let's see whether we can get a date/time probe vector

In [5]:
def gen_labeled_text(n):
    # date as text, date as utc timestamp in seconds, sample randomly from between 1990 and 2022
    start_timestamp = datetime(2013, 1, 1).timestamp()
    end_timestamp = datetime(2016, 1, 1).timestamp()
    labeled_text = []
    for i in range(n):
        timestamp = start_timestamp + (end_timestamp - start_timestamp) * random.random()
        date = datetime.fromtimestamp(timestamp)
        # date like "Monday 15th November 2021 8AM"
        text = date.strftime("Today is a %A. It's the %dth of %B, %Y. The time is %I %p. This is the point in time when")
        label = timestamp
        labeled_text.append((text, label))
    # normalize labels to have mean 0 and std 1
    labels = [label for _, label in labeled_text]
    mean = sum(labels) / len(labels)
    std = (sum((label - mean) ** 2 for label in labels) / len(labels)) ** 0.5
    labeled_text = [(text, (label - mean) / std) for text, label in labeled_text]
    return labeled_text

In [6]:
data = gen_labeled_text(10_000)
print(data[0])

("Today is a Sunday. It's the 19th of May, 2013. The time is 09 PM. This is the point in time when", -1.280076907780599)


## Training

In [7]:
probe = train_probe(
    labeled_text=data,
    model=model,
    tokenizer=tokenizer,
    layer=4,
    n_epochs=20,
    batch_size=128,
    lr=1e-2,
    save_to=None,
    token_position=-2
)

running on device: cuda:0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:06<00:00, 12.24it/s]


Epoch 0, mean loss: 5.525280564117431


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 14.07it/s]


Epoch 1, mean loss: 0.9294690269470215


...

Epoch 17, mean loss: 0.1643917332649231


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 13.80it/s]


Epoch 18, mean loss: 0.15300347843170167


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 13.77it/s]

Epoch 19, mean loss: 0.13035983095169068


## Using the vector

In [8]:
direction = probe.weight[0]
bias = probe.bias

In [9]:
bias

Parameter containing:
tensor([0.0284], device='cuda:0', requires_grad=True)

In [10]:
run_simple_steering(
    text=["The current date is"],
    model=model,
    tokenizer=tokenizer,
    layer=4,
    multiplier=-3,
    vector=direction.detach(),
    max_n_tokens=10,
    save_to=None
)

[{'input': 'The current date is',
  'output': 'The current date is the date of the first occurrence',
  'layer': 4,
  'multiplier': -3}]

In [11]:
run_simple_steering(
    text=["The current date is"],
    model=model,
    tokenizer=tokenizer,
    layer=4,
    multiplier=4,
    vector=direction.detach(),
    max_n_tokens=10,
    save_to=None
)

[{'input': 'The current date is',
  'output': 'The current date is the next year, the next',
  'layer': 4,
  'multiplier': 4}]

# CAA

## Let's get some contrast pairs

Let's try an easy direction - positive vs negative sentiment

In [12]:
GOOD = [
    "The weather is really nice",
    "I'm so happy",
    "This cake is absolutely delicious",
    "I love my friends",
    "I'm feeling great",
    "I'm so excited",
    "This is the best day ever",
    "I really like this gift",
    "Croissants are my favorite",
    "The movie was fantastic",
    "I got a promotion at work",
    "My vacation was amazing",
    "The concert exceeded my expectations",
    "I'm grateful for my family",
    "This book is incredibly engaging",
    "The restaurant service was excellent",
    "I'm proud of my accomplishments",
    "The sunset is breathtakingly beautiful",
    "I passed my exam with flying colors",
    "This coffee tastes perfect",
]

BAD = [
    "The weather is really bad",
    "I'm so sad",
    "This cake is completely inedible",
    "I hate my enemies",
    "I'm feeling awful",
    "I'm so anxious",
    "This is the worst day ever",
    "I dislike this gift",
    "Croissants are disgusting",
    "The movie was terrible",
    "I got fired from work",
    "My vacation was a disaster",
    "The concert was a huge disappointment",
    "I'm frustrated with my family",
    "This book is incredibly boring",
    "The restaurant service was horrible",
    "I'm ashamed of my mistakes",
    "The weather is depressingly gloomy",
    "I failed my exam miserably",
    "This coffee tastes awful",
]

In [13]:
dataset = [
    (text, True) for text in GOOD
] + [
    (text, False) for text in BAD
]

## Getting the CAA vectors

In [14]:
vectors = get_caa_vecs(
    labeled_text=dataset,
    model=model,
    tokenizer=tokenizer,
    layers=range(3, 8),
    save_to=None              
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 121.28it/s]


## Using the CAA vectors

In [15]:
run_simple_steering(
    text=["I think that this cat is"],
    model=model,
    tokenizer=tokenizer,
    layer=6,
    multiplier=-2,
    vector=vectors[6],
    max_n_tokens=20,
    save_to=None,
)

[{'input': 'I think that this cat is',
  'output': "I think that this cat is a bit of a mess. It's not a good cat. It",
  'layer': 6,
  'multiplier': -2}]

In [16]:
run_simple_steering(
    text=["I think that this cat is"],
    model=model,
    tokenizer=tokenizer,
    layer=6,
    multiplier=2,
    vector=vectors[6],
    max_n_tokens=20,
    save_to=None,
)

[{'input': 'I think that this cat is',
  'output': 'I think that this cat is a great addition to the collection of cats that we have in our collection',
  'layer': 6,
  'multiplier': 2}]